<a href="https://colab.research.google.com/github/shkgiri/AI-ML-for-All/blob/main/documentation/tutorials/predict_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Making predictions

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/decision_forests/tutorials/predict_colab"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/decision-forests/blob/main/documentation/tutorials/predict_colab.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/decision-forests/blob/main/documentation/tutorials/predict_colab.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/decision-forests/documentation/tutorials/predict_colab.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>




Welcome to the **Prediction Colab** for **TensorFlow Decision Forests** (**TF-DF**).
In this colab, you will learn about different ways to generate predictions with a previously trained **TF-DF** model using the **Python API**.

<i><b>Remark:</b> The Python API shown in this Colab is simple to use and well-suited for experimentation. However, other APIs, such as TensorFlow Serving and the C++ API are better suited for production systems as they are faster and more stable. The exhaustive list of all Serving APIs is available [here](https://ydf.readthedocs.io/en/latest/serving_apis.html).</i>

In this colab, you will:

1. Use the `model.predict()` function on a TensorFlow Dataset created with `pd_dataframe_to_tf_dataset`.
1. Use the `model.predict()` function on a TensorFlow Dataset created manually.
1. Use the `model.predict()` function on Numpy arrays.
1. Make predictions with the CLI API.
1. Benchmark the inference speed of a model with the CLI API.




## Important remark

The dataset used for predictions should have the **same feature names and types** as the dataset used for training. Failing to do so, will likely raise errors.

For example, training a model with two features `f1` and `f2`, and trying to generate predictions on a dataset without `f2` will fail. Note that it is okay to set (some or all) feature values as "missing". Similarly, training a model where `f2` is a numerical feature (e.g., float32), and applying this model on a dataset where `f2` is a text (e.g., string) feature will fail.

While abstracted by the Keras API, a model instantiated in Python (e.g., with
`tfdf.keras.RandomForestModel()`) and a model loaded from disk (e.g., with
`tf_keras.models.load_model()`) can behave differently. Notably, a Python
instantiated model automatically applies necessary type conversions. For
example, if a `float64` feature is fed to a model expecting a `float32` feature,
this conversion is performed implicitly. However, such a conversion is not
possible for models loaded from disk. It is therefore important that the
training data and the inference data always have the exact same type.

## Setup

First, we install TensorFlow Dececision Forests...

In [1]:
!pip install tensorflow_decision_forests

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
    

... , and import the libraries used in this example.

In [2]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

## `model.predict(...)` and `pd_dataframe_to_tf_dataset` function

TensorFlow Decision Forests implements the [Keras](https://keras.io/) model API.
As such, TF-DF models have a `predict` function to make predictions. This function  takes as input a [TensorFlow Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) and outputs a prediction array.
The simplest way to create a TensorFlow dataset is to use [Pandas](https://pandas.pydata.org/) and the the `tfdf.keras.pd_dataframe_to_tf_dataset(...)` function.

The next example shows how to create a TensorFlow dataset using `pd_dataframe_to_tf_dataset`.

In [3]:
pd_dataset = pd.DataFrame({
    "feature_1": [1,2,3],
    "feature_2": ["a", "b", "c"],
    "label": [0, 1, 0],
})

pd_dataset

,feature_1,feature_2,label
0,1,a,0
1,2,b,1
2,3,c,0


In [ ]:
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pd_dataset, label="label")

for features, label in tf_dataset:
  print("Features:",features)
  print("label:", label)

<i>**Note:** "pd_" stands for "pandas". "tf_" stands for "TensorFlow".</i>

A TensorFlow Dataset is a function that outputs a sequence of values. Those values can be simple arrays (called Tensors) or arrays organized into a structure (for example, arrays organized in a dictionary).


The following example shows the training and inference (using `predict`) on a toy dataset:

In [ ]:
# Creating a training dataset in Pandas
pd_train_dataset = pd.DataFrame({
    "feature_1": np.random.rand(1000),
    "feature_2": np.random.rand(1000),
})
pd_train_dataset["label"] = pd_train_dataset["feature_1"] > pd_train_dataset["feature_2"]

pd_train_dataset

In [ ]:
# Creating a serving dataset with Pandas
pd_serving_dataset = pd.DataFrame({
    "feature_1": np.random.rand(500),
    "feature_2": np.random.rand(500),
})

pd_serving_dataset

Let's convert the Pandas dataframes into TensorFlow datasets:

In [ ]:
tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pd_train_dataset, label="label")
tf_serving_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pd_serving_dataset)

We can now train a model on `tf_train_dataset`:

In [ ]:
model = tfdf.keras.RandomForestModel(verbose=0)
model.fit(tf_train_dataset)

And then generate predictions on `tf_serving_dataset`:

In [ ]:
# Print the first 10 predictions.
model.predict(tf_serving_dataset, verbose=0)[:10]

## `model.predict(...)` and manual TF datasets

In the previous section, we showed how to create a TF dataset using the `pd_dataframe_to_tf_dataset` function. This option is simple but poorly suited for large datasets. Instead, TensorFlow offers [several options](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) to create a TensorFlow dataset.
The next examples shows how to create a dataset using the `tf.data.Dataset.from_tensor_slices()` function.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5])

for value in dataset:
  print("value:", value.numpy())

TensorFlow models are trained with mini-batching: Instead of being fed one at a time, examples are grouped in "batches". For Neural Networks, the batch size impacts the quality of the model, and the optimal value needs to be determined by the user during training. For Decision Forests, the batch size has no impact on the model. However, for compatibility reasons, **TensorFlow Decision Forests expects the dataset to be batched**. Batching is done with the `batch()` function.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5]).batch(2)

for value in dataset:
  print("value:", value.numpy())

TensorFlow Decision Forests expects the dataset to be of one of two structures:

- features, label
- features, label, weights

The features can be a single 2 dimensional array (where each column is a feature and each row is an example), or a dictionary of arrays.

Following is an example of a dataset compatible with TensorFlow Decision Forests:

In [ ]:
# A dataset with a single 2d array.
tf_dataset = tf.data.Dataset.from_tensor_slices(
    ([[1,2],[3,4],[5,6]], # Features
    [0,1,0], # Label
    )).batch(2)

for features, label in tf_dataset:
  print("features:", features)
  print("label:", label)

In [ ]:
# A dataset with a dictionary of features.
tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({
    "feature_1": [1,2,3],
    "feature_2": [4,5,6],
    },
    [0,1,0], # Label
    )).batch(2)

for features, label in tf_dataset:
  print("features:", features)
  print("label:", label)

Let's train a model with this second option.

In [ ]:
tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({
    "feature_1": np.random.rand(100),
    "feature_2": np.random.rand(100),
    },
    np.random.rand(100) >= 0.5, # Label
    )).batch(2)

model = tfdf.keras.RandomForestModel(verbose=0)
model.fit(tf_dataset)

The `predict` function can be used directly on the training dataset:

In [ ]:
# The first 10 predictions.
model.predict(tf_dataset, verbose=0)[:10]

## `model.predict(...)` and `model.predict_on_batch()` on dictionaries

In some cases, the `predict` function can be used with an array (or dictionaries of arrays) instead of TensorFlow Dataset.

The following example uses the previously trained model with a dictionary of NumPy arrays.

In [ ]:
# The first 10 predictions.
model.predict({
    "feature_1": np.random.rand(100),
    "feature_2": np.random.rand(100),
    }, verbose=0)[:10]

In the previous example, the arrays are automatically batched. Alternatively, the `predict_on_batch` function can be used to make sure that all the examples are run in the same batch.

In [ ]:
# The first 10 predictions.
model.predict_on_batch({
    "feature_1": np.random.rand(100),
    "feature_2": np.random.rand(100),
    })[:10]


**Note:** If `predict` does not work on raw data such as in the example above, try to use the `predict_on_batch` function or convert the raw data into a TensorFlow Dataset.

## Inference with the YDF format

This example shows how to run a TF-DF model trained with the CLI API ([one of the other Serving APIs](https://ydf.readthedocs.io/en/latest/serving_apis.html)). We will also use the Benchmark tool to measure the inference speed of the model.

Let's start by training and saving a model:

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(verbose=0)
model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(pd_train_dataset, label="label"))
model.save("my_model")

Let's also export the dataset to a csv file:

In [ ]:
pd_serving_dataset.to_csv("dataset.csv")

Let's download and extract the [Yggdrasil Decision Forests](https://ydf.readthedocs.io/en/latest/index.html) CLI tools.

In [ ]:
!wget https://github.com/google/yggdrasil-decision-forests/releases/download/1.0.0/cli_linux.zip
!unzip cli_linux.zip

Finally, let's make predictions:

**Remarks:**


- TensorFlow Decision Forests (TF-DF) is based on the [Yggdrasil Decision Forests](https://ydf.readthedocs.io/en/latest/index.html) (YDF) library, and  TF-DF model always contains a YDF model internally. When saving a TF-DF model to disk, the TF-DF model directory contains an `assets` sub-directory containing the YDF model. This YDF model can be used with all [YDF tools](https://ydf.readthedocs.io/en/latest/cli_commands.html). In the next example, we will use the `predict` and `benchmark_inference` tools. See the [model format documentation](https://ydf.readthedocs.io/en/latest/convert_model.html) for more details.
- YDF tools assume that the type of the dataset is specified using a prefix, e.g. `csv:`. See the [YDF user manual](https://ydf.readthedocs.io/en/latest/cli_user_manual.html#dataset-path-and-format) for more details.

In [ ]:
!./predict --model=my_model/assets --dataset=csv:dataset.csv --output=csv:predictions.csv

We can now look at the predictions:

In [ ]:
pd.read_csv("predictions.csv")

The speed of inference of a model can be measured with the [benchmark inference](https://ydf.readthedocs.io/en/latest/benchmark_inference.html) tool.

**Note:** Prior to YDF version 1.1.0, the dataset used in the benchmark inference needs to have a `__LABEL` column.

In [ ]:
# Create the empty label column.
pd_serving_dataset["__LABEL"] = 0
pd_serving_dataset.to_csv("dataset.csv")

In [ ]:
!./benchmark_inference \
  --model=my_model/assets \
  --dataset=csv:dataset.csv \
  --batch_size=100 \
  --warmup_runs=10 \
  --num_runs=50

In this benchmark, we see the inference speed for different inference engines. For example, "time/example(us) = 0.6315" (can change in different runs) indicates that the inference of one example takes 0.63 micro-seconds. That is, the model can be run ~1.6 millions of times per seconds.

**Note:** TF-DF and the other API always automatically select the fastest inference engine available.